# Modelo no supervisado de clasificación de texto

## Introducción

En este notebook se presenta un modelo no supervisado de clasificación de texto. El modelo se basa en el uso de embeddings de palabras y clustering. Se utiliza el algoritmo de clustering KMeans para agrupar los textos en clusters. 



In [24]:
### Librerias necesarias Doc2Vec

import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

### Librerias necesarias para el preprocesamiento de texto
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

### Descargar stopwords
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.model_selection import train_test_split

### Otras librerias necesarias
import numpy as np
import pandas as pd
import re
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 1.1 Modelo Doc2Vec

El modelo Doc2Vec es una extensión del modelo Word2Vec que agrega un vector adicional para cada documento en el corpus. Este vector adicional se entrena junto con los vectores de palabras y se utiliza para representar el contenido de un documento en un espacio vectorial. Esto permite comparar documentos y encontrar similitudes entre ellos. En este caso ocurre lo siguiente:

#### ¿Cómo funciona Doc2Vec?

Doc2Vec tiene dos enfoques principales:

1. **PV-DM (Distributed Memory):** Este modelo predice una palabra en el documento usando el contexto de palabras a su alrededor y un vector que representa al documento completo. En otras palabras:
   - Se toma un documento y se le asigna un vector único (el vector del documento).
   - Luego, junto con las palabras cercanas, este vector se usa para predecir una palabra en el documento.
   - Al entrenar el modelo, este vector se va ajustando para que capture mejor el significado del documento.

2. **PV-DBOW (Distributed Bag of Words):** Aquí se usa solo el vector del documento para predecir palabras al azar dentro del documento, sin tener en cuenta el contexto de palabras. Es más rápido, pero menos preciso.

En resumen, Doc2Vec es un modelo que entrena vectores para palabras y documentos al mismo tiempo. Estos vectores pueden ser usados luego para comparar documentos, encontrar similitudes entre ellos, o incluso para clasificarlos.

![Proceso doc2vec](../../Datos/Imágenes/doc2vec.png)


## 2. Cargar el texto

El dataset "es_tweets_laboral" de la colección "somosnlp-hackathon-2022" en Hugging Face está diseñado específicamente para el análisis de texto relacionado con temas laborales en español. Este dataset contiene tuits que abordan temas laborales, y es ideal para tareas de clasificación de texto, análisis de sentimientos, y otras aplicaciones de procesamiento de lenguaje natural (NLP) enfocadas en el ámbito laboral.

**Características del Dataset:**
- *Contenido*: Incluye tuits en español relacionados con temas laborales, como empleo, condiciones de trabajo, y derechos laborales.
- *Etiquetas*: Los tuits pueden estar etiquetados según el tema o el sentimiento, lo que facilita su uso en tareas de clasificación supervisada.
- *Aplicaciones*: Este dataset es útil para construir modelos que analicen la percepción de los usuarios sobre temas laborales, detectar tendencias en el mercado laboral, o identificar problemas comunes en el ámbito laboral.

El dataset es parte de un esfuerzo colaborativo durante el Hackathon de SomosNLP en 2022, que busca fomentar el desarrollo de tecnologías de procesamiento de lenguaje natural en español.



In [25]:
Dataset = pd.read_csv("C:/Users/USER/OneDrive - universidadean.edu.co/5. NLP/Repositorio NLP/Actividad 1/reviews_booking.csv")
Dataset['Calificación'] = Dataset['Calificación'].astype(str).str.replace(',', '.').astype(float)
Dataset = Dataset.drop("Unnamed: 0", axis=1)
Dataset

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_16808\2260604972.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  Dataset = pd.read_csv("C:/Users/USER/OneDrive - universidadean.edu.co/5. NLP/Repositorio NLP/Actividad 1/reviews_booking.csv")


,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña
0,Venezuela,NaN,2 noches,julio de 2023,En pareja,31 de julio de 2023,Excepcional,10.0,Fue la Mejor Opción que pudimos haber tomado e...,NaN,Sí
1,Colombia,NaN,4 noches,octubre de 2024,Persona que viaja sola,23 de octubre de 2024,Fantástica,10.0,"Ubicación, comodidades y limpieza",NaN,Sí
2,Colombia,NaN,1 noche,octubre de 2024,Persona que viaja sola,15 de octubre de 2024,Excepcional,10.0,"El lugar esta muy bien ubicado, el apartamento...",NaN,Sí
3,Colombia,NaN,1 noche,agosto de 2024,En pareja,20 de agosto de 2024,Gran apartamento en una gran ubicación.,10.0,Todo estuvo perfecto. La ubicación y la relaci...,NaN,Sí
4,Colombia,NaN,3 noches,marzo de 2024,En pareja,27 de marzo de 2024,Excepcional,10.0,"Es muy cómodo y acogedor, la atención es excel...",NaN,Sí
...,...,...,...,...,...,...,...,...,...,...,...
438208,Estados Unidos,Apartamento Suite,1 noche,septiembre de 2024,En familia,22 de octubre de 2024,Fantástico,9.0,NaN,NaN,No
438209,Colombia,Apartamento de 1 dormitorio,1 noche,octubre de 2024,Persona que viaja sola,9 de octubre de 2024,Excepcional,10.0,NaN,NaN,No
438210,México,Apartamento de 1 dormitorio,4 noches,agosto de 2024,Persona que viaja sola,6 de septiembre de 2024,Pésimo,1.0,No me pareció nada,No obstante que dos días antes de mi llegada a...,Sí
438211,Colombia,Apartamento de 1 dormitorio,2 noches,julio de 2024,En pareja,22 de julio de 2024,Excepcional,10.0,NaN,NaN,No


In [26]:
# Crear una lista de stopwords
stop_words = set(stopwords.words('spanish'))

#Función para contar stopwords en un texto
def count_stopwords(text):
    if not isinstance(text, str):
        return 0
    words = text.split()
    return sum(1 for word in words if word.lower() in stop_words)

# Contar stopwords en la columna 'Cosas Positivas'
stopword_counts = Dataset['Cosas Positivas'].apply(count_stopwords)

# Filtrar filas con menos de 5 stopwords
Corpus_positivo = Dataset[stopword_counts >= 15]

# Filtrar filas donde 'Cosas Positivas' tenga más de #### palabra
Corpus_positivo = Corpus_positivo[Corpus_positivo['Cosas Positivas'].str.split().str.len() > 30]

# Filtrar filas donde 'Calificación' sea mayor a 9
Corpus_positivo = Corpus_positivo[Corpus_positivo['Calificación'] > 9]

Corpus_positivo

,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña
0,Venezuela,NaN,2 noches,julio de 2023,En pareja,31 de julio de 2023,Excepcional,10.0,Fue la Mejor Opción que pudimos haber tomado e...,NaN,Sí
17,Ecuador,NaN,5 noches,octubre de 2024,Persona que viaja sola,29 de octubre de 2024,Excepcional,10.0,Me gusto Poder comprar los alimentos a mi gust...,NaN,Sí
70,Colombia,NaN,1 noche,agosto de 2022,En familia,9 de agosto de 2022,"Maravilloso alojamiento, recomendado!",10.0,"La calidad de personas es maravilloso, desde q...",NaN,Sí
476,Ecuador,NaN,1 noche,agosto de 2024,Persona que viaja sola,26 de agosto de 2024,Excepcional,10.0,La atención como me recibieron es muy buena po...,NaN,Sí
485,Colombia,NaN,2 noches,junio de 2024,En familia,4 de julio de 2024,Un hotel muy agradable,10.0,"El hotel es muy acogedor, está ubicado en una ...",La verdad no hubo nada que no me haya gustado.,Sí
...,...,...,...,...,...,...,...,...,...,...,...
438075,Colombia,Apartamento de 1 dormitorio,5 noches,noviembre de 2024,Persona que viaja sola,10 de noviembre de 2024,Súper,10.0,"El apartamento tiene vista muy bonita, puedes ...",NaN,Sí
438081,Colombia,Habitación Individual,1 noche,junio de 2024,Persona que viaja sola,2 de junio de 2024,Excelente y buen precio,10.0,El mejor hotel boutique en el que me he quedad...,Todo estuvo muy bien.,Sí
438083,Colombia,Habitación Individual,1 noche,junio de 2024,Persona que viaja sola,2 de junio de 2024,Confortable y Conveniente,10.0,"El personal fue amable y atento, la habitación...",NaN,Sí
438170,Colombia,Apartamento de 1 dormitorio,11 noches,julio de 2024,En familia,5 de agosto de 2024,Excelente,10.0,Excelente ubicación con muchos museos y zonas ...,NaN,Sí


In [27]:
Corpus_positivo = Corpus_positivo['Cosas Positivas']

In [36]:
# Dividir el dataset en entrenamiento y prueba (sin variable objetivo)
train_df, test_df = train_test_split(Corpus_positivo, test_size=0.2, random_state=42)

# Verificar las dimensiones
print("Shape of X_train:", train_df.shape)
print("Shape of X_test:", test_df.shape)

Shape of X_train: (4611,)
Shape of X_test: (1153,)


In [38]:
train_df = train_df.to_frame()  # Convierte la Series en DataFrame
test_df = test_df.to_frame()
train_df

,Cosas Positivas
321638,El personal es muy atento y a comedido son per...
396918,La ubicación es de lo mejor. Te queda cerca lo...
352250,La amabilidad de las caseras de igual forma de...
142026,"La atención fue maravillosa, personal muy amab..."
226494,"El hotel esta ubicado en un area muy segura, p..."
...,...
279707,Siempre atienden con la mejor disposición y am...
391490,El desayuno es espectacular; su personal muy a...
396866,Me gustó su centralidad y facilidad para poder...
408710,El hostal es muy agreable. Tiene areas differe...


In [39]:
print(type(Corpus_positivo))
print(Corpus_positivo.keys())  # Si es un diccionario, mostrará las claves disponibles


<class 'pandas.core.series.Series'>
Index([     0,     17,     70,    476,    485,    512,    519,    554,    575,
          598,
       ...
       437983, 437994, 438017, 438030, 438058, 438075, 438081, 438083, 438170,
       438178],
      dtype='int64', length=5764)


In [40]:
# Verificar las dimensiones
print("Shape of train data: ", train_df.shape)
print("Shape of test data: ", test_df.shape)

# Mostrar los primeros registros del conjunto de entrenamiento
print(train_df.head())


train_df.head()

Shape of train data:  (4611, 1)
Shape of test data:  (1153, 1)
                                          Cosas Positivas
321638  El personal es muy atento y a comedido son per...
396918  La ubicación es de lo mejor. Te queda cerca lo...
352250  La amabilidad de las caseras de igual forma de...
142026  La atención fue maravillosa, personal muy amab...
226494  El hotel esta ubicado en un area muy segura, p...


,Cosas Positivas
321638,El personal es muy atento y a comedido son per...
396918,La ubicación es de lo mejor. Te queda cerca lo...
352250,La amabilidad de las caseras de igual forma de...
142026,"La atención fue maravillosa, personal muy amab..."
226494,"El hotel esta ubicado en un area muy segura, p..."


In [41]:
test_df.head()

,Cosas Positivas
297824,"El alojamiento es Perfecto, pero está muy mal ..."
342352,Claudia la recepcionista nos hizo sentir como ...
150516,El personal es muy amable y hace todo lo posib...
277897,"La atención es muy agradable, tiene unas vista..."
350542,La atención de cada uno de los chicos del pers...


## 3. Preprocesamiento

Empezamos por cargar el dataset y realizar un preprocesamiento básico de los textos. En este caso, se eliminan las menciones a usuarios, los enlaces, y los caracteres especiales. Además, se convierten los textos a minúsculas y se eliminan las stopwords.

In [42]:
########## Preprocesamiento de texto ###############

def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    return text

# Aplicando la función de preprocesamiento a los datos de entrenamiento y prueba

train_df['text_pre'] = train_df['Cosas Positivas'].apply(preprocess_text)

test_df['text_pre'] = test_df['Cosas Positivas'].apply(preprocess_text)

train_df.head()

,Cosas Positivas,text_pre
321638,El personal es muy atento y a comedido son per...,el personal es muy atento y a comedido son per...
396918,La ubicación es de lo mejor. Te queda cerca lo...,la ubicación es de lo mejor. te queda cerca lo...
352250,La amabilidad de las caseras de igual forma de...,la amabilidad de las caseras de igual forma de...
142026,"La atención fue maravillosa, personal muy amab...","la atención fue maravillosa, personal muy amab..."
226494,"El hotel esta ubicado en un area muy segura, p...","el hotel esta ubicado en un area muy segura, p..."


## 4. Modelo de Clasificación de Texto

Una vez que hemos preprocesado los textos, podemos aplicar un modelo de clasificación no supervisado para agruparlos en categorías o clusters. En este caso, utilizaremos el algoritmo de clustering KMeans para agrupar los textos en clusters. AUnque primero debemos convertir los textos en vectores numéricos utilizando embeddings de palabras.


In [43]:
################## Entrenamiento de Word2Vec ####################

# Tokenizando el texto

train_df['text_tokens'] = train_df['text_pre'].apply(lambda x: x.split())
test_df['text_tokens'] = test_df['text_pre'].apply(lambda x: x.split())

train_df.head()

,Cosas Positivas,text_pre,text_tokens
321638,El personal es muy atento y a comedido son per...,el personal es muy atento y a comedido son per...,"[el, personal, es, muy, atento, y, a, comedido..."
396918,La ubicación es de lo mejor. Te queda cerca lo...,la ubicación es de lo mejor. te queda cerca lo...,"[la, ubicación, es, de, lo, mejor., te, queda,..."
352250,La amabilidad de las caseras de igual forma de...,la amabilidad de las caseras de igual forma de...,"[la, amabilidad, de, las, caseras, de, igual, ..."
142026,"La atención fue maravillosa, personal muy amab...","la atención fue maravillosa, personal muy amab...","[la, atención, fue, maravillosa,, personal, mu..."
226494,"El hotel esta ubicado en un area muy segura, p...","el hotel esta ubicado en un area muy segura, p...","[el, hotel, esta, ubicado, en, un, area, muy, ..."


In [44]:
# Entrenando el modelo Doc2Vec

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(train_df['text_tokens'])]

# Definiendo el modelo Doc2Vec

model = Doc2Vec(documents, vector_size=100, window=2, min_count=1, workers=4)

# Guardando el modelo entrenado

model.save("doc2vec.model")


In [52]:
# Cargando el modelo entrenado

model = Doc2Vec.load("doc2vec.model")

# Obteniendo el vector de una palabra

model.wv['hotel']

# Obteniendo las palabras más similares a una palabra

model.wv.most_similar('servicio')


[('trato', 0.91214919090271),
 ('servicio,', 0.9041281342506409),
 ('desayuno,', 0.8980856537818909),
 ('aseo', 0.8912537693977356),
 ('restaurante', 0.8795363903045654),
 ('precio,', 0.8715824484825134),
 ('servicio.', 0.8710347414016724),
 ('desayuno.', 0.8664168119430542),
 ('equipo', 0.8608136177062988),
 ('mantenimiento', 0.8566083312034607)]

In [53]:
# Obteniendo la similitud entre dos palabras

model.wv.similarity('trabajo', 'empleo')


KeyError: "Key 'empleo' not present"

In [ ]:
#### Hacemos un clustering de los tweets con KMeans

from sklearn.cluster import KMeans

# Obteniendo los vectores de los tweets

vectors = [model.infer_vector(doc) for doc in train_df['text_tokens']]

# Definiendo el modelo KMeans

kmeans = KMeans(n_clusters=5, random_state=0)

# Entrenando el modelo KMeans

kmeans.fit(vectors)

# Obteniendo las etiquetas de los clusters

train_df['cluster'] = kmeans.labels_

# Explorando los clusters

train_df['cluster'].value_counts()



cluster
3    55
4    37
0    37
2    29
1    26
Name: count, dtype: int64

In [ ]:
### Veamos los tweets de un cluster

pd.set_option('display.max_colwidth', None)

train_df[train_df['cluster'] == 0]['text'].head(10)

5                                                                                                                            es un poco injusto que haya gente que batalla para tener un buen salario o trabajo y que haya gente que esté bien acomodada y viviendo bien sin necesidad de estar batallando
8                                                                                                                                                                                 Dos profesoras e investigadoras de la Universidad del Istmo #Unistmo, campus Ixtepec, denunciaron violencia laboral y h…
10                                                                                                                                                                       los #bomberosforestales siguen su lucha! por unas #condicioneslaborales dignas. por no tener que lamentarse tarde. soluciones ya!
13                                                                       @CarolinaPiparo Carolina...los

In [ ]:
### Veamos el centroide de cada cluster

for i in range(5):
    print("Cluster ", i)
    print(model.wv.most_similar(positive=[kmeans.cluster_centers_[i]], topn=10))
    print("\n\n")

Cluster  0
[('tv', 0.3773950934410095), ('morena', 0.35579192638397217), ('salario', 0.3412707448005676), ('medida', 0.32781466841697693), ('quedar', 0.32356029748916626), ('hoy', 0.2996211349964142), ('salum', 0.29874157905578613), ('si', 0.2883167862892151), ('superintendencia', 0.28237178921699524), ('recae', 0.28189268708229065)]



Cluster  1
[('si', 0.31819480657577515), ('presidenta', 0.303609699010849), ('ls', 0.3002225160598755), ('planteamos', 0.29824796319007874), ('ahorra', 0.2975274622440338), ('sacando', 0.2972254753112793), ('necesitan', 0.29635271430015564), ('salum', 0.284090131521225), ('locales', 0.2769164443016052), ('salario', 0.2698799669742584)]



Cluster  2
[('vida', 0.34470483660697937), ('q', 0.3345752954483032), ('trabajo', 0.32723310589790344), ('región', 0.31122347712516785), ('todas', 0.300120085477829), ('firmes', 0.2965523898601532), ('misoginia', 0.2937416136264801), ('diciendo', 0.28792816400527954), ('universidad', 0.286060094833374), ('importante', 